In [1]:
from tensorflow.keras.layers import Dense, Input, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import resnet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
batch_size = 64
width, height, channel = 400, 400, 3
no_epochs = 30
verbosity = 2
input_shape = (width, height, channel)

In [3]:
# ResNet50 모델 사용

pretrained_model = tf.keras.applications.InceptionResNetV2(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=1000
)

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('preprocessed_data/train',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory('preprocessed_data/test',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

pretrained_model.trainable=True
set_trainable=False

for layer in pretrained_model.layers:
    if layer.name == 'block7c_project_conv':
        set_trainable=True
    if set_trainable:
        layer.trainable=True
    else:
        layer.trainable=False


model=Sequential()
model.add(pretrained_model)
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[metrics.AUC(), 'accuracy'])

Found 2603 images belonging to 2 classes.
Found 386 images belonging to 2 classes.


In [5]:
es_callback = EarlyStopping(monitor='val_auc', mode='max', patience=8,
                                              verbose=1, min_delta=0.0001, restore_best_weights=True)

history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples//batch_size,
                    epochs = no_epochs,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//batch_size,
                    callbacks= [es_callback],
                    verbose=verbosity,
                   )

Epoch 1/30
40/40 - 36s - loss: 0.6457 - auc: 0.8607 - accuracy: 0.7901 - val_loss: 0.3973 - val_auc: 0.9932 - val_accuracy: 0.7891
Epoch 2/30
40/40 - 27s - loss: 0.1116 - auc: 0.9945 - accuracy: 0.9642 - val_loss: 0.0462 - val_auc: 0.9999 - val_accuracy: 0.9922
Epoch 3/30
40/40 - 27s - loss: 0.0537 - auc: 0.9997 - accuracy: 0.9933 - val_loss: 0.0667 - val_auc: 1.0000 - val_accuracy: 0.9922
Epoch 4/30
40/40 - 26s - loss: 0.0352 - auc: 1.0000 - accuracy: 0.9980 - val_loss: 0.0204 - val_auc: 1.0000 - val_accuracy: 1.0000
Epoch 5/30
40/40 - 26s - loss: 0.0264 - auc: 1.0000 - accuracy: 0.9988 - val_loss: 0.0235 - val_auc: 1.0000 - val_accuracy: 1.0000
Epoch 6/30
40/40 - 27s - loss: 0.0201 - auc: 1.0000 - accuracy: 0.9992 - val_loss: 0.0175 - val_auc: 1.0000 - val_accuracy: 1.0000
Epoch 7/30
40/40 - 28s - loss: 0.0156 - auc: 1.0000 - accuracy: 0.9996 - val_loss: 0.0101 - val_auc: 1.0000 - val_accuracy: 1.0000
Epoch 8/30
40/40 - 27s - loss: 0.0128 - auc: 1.0000 - accuracy: 1.0000 - val_loss: 

In [6]:
prediction = model.predict(valid_generator)

In [7]:
import numpy as np

print(len(prediction[np.where(prediction > 0.5)]))
print(len(prediction[np.where(prediction < 0.5)]))

317
69


In [8]:
prediction

array([[8.78724217e-01],
       [1.49441315e-02],
       [8.94827127e-01],
       [9.06180799e-01],
       [1.00000000e+00],
       [1.00000000e+00],
       [6.10144615e-01],
       [9.46370602e-01],
       [9.97664094e-01],
       [9.99259651e-01],
       [1.00000000e+00],
       [9.75715101e-01],
       [8.76153290e-01],
       [9.87848163e-01],
       [9.99998927e-01],
       [9.99508858e-01],
       [8.70421052e-01],
       [9.91302609e-01],
       [5.51492767e-03],
       [9.99931455e-01],
       [9.24107611e-01],
       [8.65700841e-01],
       [1.00000000e+00],
       [9.96612012e-01],
       [9.86390054e-01],
       [9.99704063e-01],
       [9.55924451e-01],
       [8.90985364e-04],
       [8.47343922e-01],
       [9.15072501e-01],
       [9.37274754e-01],
       [1.00000000e+00],
       [1.00000000e+00],
       [9.98533368e-01],
       [6.21731440e-03],
       [4.29488398e-04],
       [1.00000000e+00],
       [9.07746434e-01],
       [1.05949631e-02],
       [9.94859099e-01],


In [9]:
tf.__version__

'2.3.1'

In [10]:
model.evaluate(valid_generator)

7/7 [==============================] - 3s 465ms/step - loss: 0.0668 - auc: 1.0000 - accuracy: 0.9922


[0.0667717382311821, 1.0, 0.9922279715538025]